### Lab 7: Sistema de Recomendacion

In [25]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import json

%matplotlib inline

### Datasets

In [27]:
ratings = pd.read_csv("ratings_small.csv")
movies = pd.read_csv("movies_metadata.csv")
links = pd.read_csv("links_small.csv")

print(ratings.shape, movies.shape, links.shape)

(100004, 4) (45466, 24) (9125, 3)


/Users/raulcastellanos/Desktop/Universidad/S7/MLModels/venvMLM/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Descripcion de los datasets

In [28]:
print("Dataset Ratings")
print("\n")
print(ratings.dtypes)
print("\n")
print("Movies Ratings")
print("\n")
print(movies.dtypes)
print("\n")
print("Links")
print("\n")
print(links.dtypes)

Dataset Ratings


userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object


Movies Ratings


adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object


Links


movieId      int64
imdbId       int64
tmdbId     

In [29]:
print(ratings.head())

   userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205


In [30]:
print(movies.head())

   adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497         

In [31]:
print(links.head())

   movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0


### Datos en la columna de 'Genres' como JSon

para la limpieza utilicé este link como referencia
https://towardsdatascience.com/cleaning-and-extracting-json-from-pandas-dataframes-f0c15f93cb38
github
https://github.com/mpHarm88/projects/tree/master/json

In [32]:
movies['genres'] = movies['genres'].str.replace("\'","\"")
jgenres = ["genres"]

def new_json(x):
    return json.loads(x)

for x in jgenres:
    movies[x] = movies[x].apply(new_json)

    
def new_json2(x):

    ls = []

    for y in range(len(x[0])):

        for k, v in x[0][y].items():
            ls.append(str(k)+ "_" +str(v))
   
    for z in range(len(ls)):

        if ls[z] not in movies.columns:
            movies[ls[z]] = 0
            movies[ls[z]].iloc[x.name] = 1
        else:
            movies[ls[z]].iloc[x.name] = 1
    return

for x in jgenres:
    movies[[x]].apply(new_json2, axis=1)

In [33]:
print(movies.columns)
print(movies["name_Animation"])

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'id_16', 'name_Animation', 'id_35',
       'name_Comedy', 'id_10751', 'name_Family', 'id_12', 'name_Adventure',
       'id_14', 'name_Fantasy', 'id_10749', 'name_Romance', 'id_18',
       'name_Drama', 'id_28', 'name_Action', 'id_80', 'name_Crime', 'id_53',
       'name_Thriller', 'id_27', 'name_Horror', 'id_36', 'name_History',
       'id_878', 'name_Science Fiction', 'id_9648', 'name_Mystery', 'id_10752',
       'name_War', 'id_10769', 'name_Foreign', 'id_10402', 'name_Music',
       'id_99', 'name_Documentary', 'id_37', 'name_Western', 'id_10770',
       'name_TV Movie', 'name_Carousel Productions', 'id_11176'

### Transformacion de los ID's para poder hacer el join

In [34]:
links['tmdbId'] = links['tmdbId'].astype(str)
links['tmdbId'] = links['tmdbId'].str.replace("\.0","")
links['tmdbId'] = links['tmdbId'].astype(object)
links.head

<ipython-input-34-f3f167569544>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  links['tmdbId'] = links['tmdbId'].str.replace("\.0","")


<bound method NDFrame.head of       movieId   imdbId  tmdbId
0           1   114709     862
1           2   113497    8844
2           3   113228   15602
3           4   114885   31357
4           5   113041   11862
...       ...      ...     ...
9120   162672  3859980  402672
9121   163056  4262980  315011
9122   163949  2531318  391698
9123   164977    27660  137608
9124   164979  3447228  410803

[9125 rows x 3 columns]>

### Join de la tabla movies y links, por medio del id

In [35]:
movie_link = pd.merge(movies,links, left_on = 'id', right_on = 'tmdbId')
movie_link.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,id_17161,name_Pulser Productions,id_18012,name_Rogue State,id_18013,name_The Cartel,id_23822,movieId,imdbId,tmdbId
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,0,0,0,0,0,0,0,1,114709,862
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,0,0,0,0,0,0,0,2,113497,8844
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0,0,0,0,0,0,0,3,113228,15602
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,0,0,0,0,0,0,0,4,114885,31357
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,0,0,0,0,0,0,0,5,113041,11862


### Join de movies con sus ratings

In [38]:
ratings1 = ratings.drop('timestamp', axis = 1)
ratings1.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [42]:
movies_data = pd.merge(movie_link, ratings1, on = 'movieId')

#rellenamos los NA's
movies_data = movies_data.fillna(0)
movies_data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,id_18012,name_Rogue State,id_18013,name_The Cartel,id_23822,movieId,imdbId,tmdbId,userId,rating
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,0,0,0,0,0,1,114709,862,7,3.0
1,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,0,0,0,0,0,1,114709,862,9,4.0
2,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,0,0,0,0,0,1,114709,862,13,5.0
3,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,0,0,0,0,0,1,114709,862,15,2.0
4,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,0,0,0,0,0,1,114709,862,19,3.0


### Variable X

In [47]:
X0 = movies_data.filter(regex=("^name"))
X1 = pd.concat([movies_data['movieId'], X0], axis = 1)
X = X1.pivot_table(index='movieId',aggfunc='mean')
X.shape

(9025, 32)

### Variable Y

In [48]:
y0 = movies_data[['userId', 'movieId', 'rating']]
y = y0.pivot_table(index='movieId',columns='userId',values='rating', aggfunc=np.mean)
y.shape

(9025, 671)

### Conversion a numpy array y data type a float

In [51]:
X_np = X.to_numpy()
y_np = np.nan_to_num(y.to_numpy())

X_np = Xnp.astype(float)
y_np = ynp.astype(float)

### Train, Test, CV

In [52]:
X_train, X_cv, X_test = np.split(X_np, [int(len(X_np)*0.7), int(len(X_np)*0.85)])
y_train, y_cv, y_test = np.split(y_np, [int(len(y_np)*0.7), int(len(y_np)*0.85)])

In [54]:
m,n = X_train.shape
z,w = y_train.shape
theta_0 = np.random.rand(w,n)

In [62]:
def cost(Y, X, T):
    return(((X @ T.T - Y) ** 2) * ((Y != 0) * 1)).sum()


def gradient(Y, X, T):
    R = (Y != 0) * 1
    hip_error = (X @ T.T - Y) * R

    return (
        hip_error @ T,
        hip_error.T @ X,
    )


def adam(
        Y,
        Xo,
        To,
        fun,
        jac,
        alpha=0.001,
        beta1=0.9,
        beta2=0.999,
        epsilon=0.0000001,
        max_iter=1000
):
    xm = np.zeros(Xo.shape)
    tm = np.zeros(To.shape)

    xv = np.zeros(Xo.shape)
    tv = np.zeros(To.shape)

    X, T, t = Xo, To, 0

    cost = []
    while t < max_iter:
        t += 1

        xg, tg = jac(Y, X, T)

        cost.append(fun(Y, X, T).item())
        #print(f'{t} \t loss={fun(Y, X, T).item():,.2f}')

        xm = beta1 * xm + (1 - beta1) * xg
        tm = beta1 * tm + (1 - beta1) * tg

        xv = beta2 * xv + (1 - beta2) * xg * xg
        tv = beta2 * tv + (1 - beta2) * tg * tg

        xmh = xm / (1 - beta1 ** t)
        tmh = tm / (1 - beta1 ** t)

        xvh = xv / (1 - beta2 ** t)
        tvh = tv / (1 - beta2 ** t)

        X -= alpha * xmh / (np.sqrt(xvh) + epsilon)
        T -= alpha * tmh / (np.sqrt(tvh) + epsilon)

    return X, T, cost

### Resultados

In [63]:
X_train_results, Tetha_train_results, Cost_train = adam(y_train, X_train, theta_0, cost, gradient)

In [64]:
Cost_train[-1]

6483.743688557745

### Cross Validation

In [65]:
Cost_cv = cost(y_cv, X_cv, Tetha_train_results)
Cost_cv

39928.719151080215

### Test

In [67]:
Cost_test = cost(y_test, X_test, Tetha_train_results)
Cost_test

22548.871311289997

### Conslusion

Al tener un costo de cross validation y test bajos esto nos ayuda a ver que el modelo puede comportarse de una manera normal ya que la variabilidad en los costos no es muy alta